## Libraries to install and Import

In [ ]:
!pip install PyPDF2 tabula-py

In [ ]:
from PyPDF2 import PdfReader
from tabula import read_pdf
import pandas as pd
import numpy as np
import re

## Refine data version 1

In [ ]:
def refine(df):
    df['new']=''
    df.fillna('', inplace=True)
    for i in df.columns[:-1]:
        df[i] = df[i].astype('str')
        df[i] = df[i].str.replace('.', '')
        df[i] = df[i].str.strip()
        df[i] = df[i].str.replace('Photo is', '')
        df[i] = df[i].str.replace('Available', '')
        df[i] = df[i].str.replace('|', '')
        df[i] = df[i].str.replace(';', '')
        df[i] = df[i].str.replace('/', '')
        df[i] = df[i].str.replace('vailable', '')
        df[i] = df[i].str.replace('$', 'S')
        df['new']+=df[i]+" "
    df.drop(columns=df.columns[:-1],inplace=True)
    return df

## Refine data version 2

In [ ]:
def refine_v2(df2):
    for i in range(len(df2)):
        temp = df2["new"].loc[i]
        temp = temp.strip()
        if all([(i not in temp) for i in ["Nl ame","lame","Name","Husband", "Father","Mother","House","Number","Age","Gender"]]) and not "".join(re.findall(r'\d+',temp)).isdigit() and temp!="":
            df2.iloc[i-1] += " "+temp
            df2.iloc[i] = ""
            df2 = df2.replace("",np.nan)
        elif len(temp)<5:
            df2.iloc[i] = ""
            df2 = df2.replace("",np.nan)
        elif temp.isalnum() and temp.isdigit():
            df2.iloc[i] = ""
            df2 = df2.replace("",np.nan)
    df2.dropna(inplace=True)
    df2['new'] = df2['new'].str.strip()
    df2 = df2.replace("",np.nan)
    df2.dropna(inplace=True)
    return df2.reset_index(drop=True)

## Create dataframe from extracted data

In [ ]:
def create_dataframe(new_df):
    res={'S. No.':[],'voter id':[],'Name':[],"Father's Name":[],"Mother's Name":[],"Husband's Name":[],"F/M/H Name":[],'House Number':[],"Gender":[],"Age":[]}
    for i in range(len(new_df)):
        l=[]
        if (i%5==0):
            l=new_df.loc[i]['new'].split()
            if len(l)>=2:
                res['S. No.']+=[l[0]]
                res['voter id']+=[l[-1]]
            elif len(l[0])<=10:
                res['S. No.']+=[l[0]]
                res['voter id']+=['Not Available']
            else:
                res['voter id']+=[l[0][:10]]
                res['S. No.']+=['Not Available']
        elif (i%5==1):
            if ":" in new_df.loc[i]['new']:
                if 'Name' in new_df.loc[i]['new']:
                  res['Name']+=[new_df.loc[i]['new'].split('Name')[1].split(':')[1].strip()]
                else:
                  res['Name']+=[new_df.loc[i]['new']]
            else:
                res['Name']+=[new_df.loc[i]['new'].split('Name')[1].strip()[1:]]
                
        elif (i%5==2):
            if "Father" in new_df.loc[i]['new']:
                l=new_df.loc[i]['new'].split("Father's Name")
                if len(l)==2:
                    if ":" in l[1]:
                      res["Father's Name"]+=[l[1].split(':')[1].strip()]
                    else:
                      res["Father's Name"]+=[l[1]]
                    res["Husband's Name"]+=["Not Available"]
                    res["Mother's Name"]+=["Not Available"]
                    res["F/M/H Name"]+=["Not Available"]
            elif "Husband" in new_df.loc[i]['new']:
                l=new_df.loc[i]['new'].split("Husband's Name")
                if  ":" in l[1]:
                    res["Husband's Name"]+=[l[1].split(':')[1].strip()]
                else:
                    res["Husband's Name"]+=[l[1].strip()[1:]]
                    
                res["Father's Name"]+=["Not Available"]
                res["Mother's Name"]+=["Not Available"]
                res["F/M/H Name"]+=["Not Available"]


            elif "Mother" in new_df.loc[i]['new']:
                l=new_df.loc[i]['new'].split("Mother's Name")
                if  ":" in l[1]:
                    res["Mother's Name"]+=[l[1].split(':')[1].strip()]
                else:
                    res["Mother's Name"]+=[l[1].strip()[1:]]
                res["Husband's Name"]+=["Not Available"]
                res["Father's Name"]+=["Not Available"]
                res["F/M/H Name"]+=["Not Available"]
           
            else:
              res["F/M/H Name"]+=[new_df.loc[i]['new']]
              res["Mother's Name"]+=["Not Available"]
              res["Husband's Name"]+=["Not Available"]
              res["Father's Name"]+=["Not Available"]
        elif (i%5==3):
            res['House Number']+=[new_df.loc[i]['new'].split(":")[-1]]
        elif (i%5==4):
            l=new_df.loc[i]['new'].split(":")
            res["Gender"]+=[l[-1]]
            if len(re.findall(r'\d+',l[1]))==0:
                res["Age"]+=['NOT FOUND']
            else:
                res["Age"]+=re.findall(r'\d+',l[1])

    return pd.DataFrame(res)

## Extract assembly details

In [ ]:
def extract_assmbly(df):
    res={'Assembly Constituency No and Name':[],'Section No and Name':[]}
    if len(df)==2:
        if len(str(df.iloc[0][0]).split(':'))>1:
            res['Assembly Constituency No and Name']+=[str(df.iloc[0][0]).split(':')[1].strip()]
        else:
            st_a=str(df.iloc[0][0]).lower()
            st_a=st_a.replace('assembly constituency no and name','').strip()
            res['Assembly Constituency No and Name']+=[st_a]
        if len(str(df.iloc[1][0]).split(':'))>1:
            res['Section No and Name']+=[str(df.iloc[1][0]).split(':')[1].strip()]
        else:
            st_s=str(df.iloc[1][0]).lower()
            st_s=st_s.replace('section no and name','').strip()
            res['Section No and Name']+=[st_s]
    else:
        if len(str(df.iloc[0][0]).split(':'))>1:
            res['Assembly Constituency No and Name']+=[str(df.iloc[0][0]).split(':')[1].strip()]
        else:
            res['Assembly Constituency No and Name']+=[str(df.iloc[0][0]).strip()]
        res['Section No and Name']+=["Not Available"]
    return pd.DataFrame(res)

In [ ]:
def page_data(start , end,path):

    df = pd.DataFrame()
    new= pd.DataFrame()
    COLUMN_NAMES=['S. No.','voter id','Name',"Father's Name","Husband's Name","F/M/H Name","House Number","Gender","Age","Assembly Constituency No and Name","Section No and Name"]
    for i in range(start,end+1):
        print(i)
        assm_poll=read_pdf(path,pages=f"{i}",pandas_options={'header':None},area=[0,0,40,300])[0]
        assm_poll=extract_assmbly(assm_poll)
        
        df_p4_1=read_pdf(path,pages=f"{i}",pandas_options={'header':None},area=[40,20,800,200])
        df_p4_2=read_pdf(path,pages=f"{i}",pandas_options={'header':None},area=[40,200,800,380])
        df_p4_3=read_pdf(path,pages=f"{i}",pandas_options={'header':None},area=[40,380,800,555])
        

        try:
            check_last=df_p4_1[0].copy()
            check_last=refine(check_last)
            if'list' in str(check_last['new'][0]+check_last['new'][1]).lower():
                break
        except:
            print("***********************************Error*****************************")
        if df_p4_3!=[]:   
            try :
                df_p4_3 = refine(df_p4_3[0])      
                df_p4_3 = refine_v2(df_p4_3)
                df_p4_3 = create_dataframe(df_p4_3)
                df_p4_3['Assembly Constituency No and Name'] = assm_poll['Assembly Constituency No and Name'][0]
                df_p4_3['Section No and Name'] = assm_poll['Section No and Name'][0]
            except :
                df_p4_3 = pd.DataFrame(columns=COLUMN_NAMES)
            
        if df_p4_2!=[]:
            try:
                df_p4_2 = refine(df_p4_2[0])        
                df_p4_2 = refine_v2(df_p4_2)
                df_p4_2 = create_dataframe(df_p4_2)
                df_p4_2['Assembly Constituency No and Name'] = assm_poll['Assembly Constituency No and Name'][0]
                df_p4_2['Section No and Name'] = assm_poll['Section No and Name'][0]
            except:
                df_p4_2 = pd.DataFrame(columns=COLUMN_NAMES)

        
        if df_p4_1!=[]:
            try:
                df_p4_1 = refine(df_p4_1[0])        
                df_p4_1 = refine_v2(df_p4_1)
                df_p4_1 = create_dataframe(df_p4_1)
                df_p4_1['Assembly Constituency No and Name'] = assm_poll['Assembly Constituency No and Name'][0]
                df_p4_1['Section No and Name'] = assm_poll['Section No and Name'][0]
            except:
                df_p4_1 = pd.DataFrame(columns=COLUMN_NAMES)

        for i in [df_p4_1 , df_p4_2 , df_p4_3]:
            if not isinstance(i,list):
              df = df.append(i,ignore_index=True)
        
    return df

In [ ]:
def page_data_3(path):
  
    assm=read_pdf(path,pages="3",pandas_options={'header':None},area=[0,0,40,300])[0]
    assm=extract_assmbly(assm)
    
    df_p4_1=read_pdf(path,pages="3",pandas_options={'header':None},area=[50,20,800,200])
    df_p4_2=read_pdf(path,pages="3",pandas_options={'header':None},area=[50,200,800,380])
    df_p4_3=read_pdf(path,pages="3",pandas_options={'header':None},area=[50,380,800,560])
    COLUMN_NAMES=['S. No.','voter id','Name',"Father's Name","Husband's Name","F/M/H Name","House Number","Gender","Age","Assembly Constituency No and Name","Section No and Name"]
    try :
        df_p4_3 = refine(df_p4_3[0])      
        df_p4_3 = refine_v2(df_p4_3)
        df_p4_3 = create_dataframe(df_p4_3)
    except :
        df_p4_3 = pd.DataFrame(columns=COLUMN_NAMES)
    try:
        df_p4_2 = refine(df_p4_2[0])        
        df_p4_2 = refine_v2(df_p4_2)
        df_p4_2 = create_dataframe(df_p4_2)
    except:
        df_p4_2 = pd.DataFrame(columns=COLUMN_NAMES)
    
    try:
        df_p4_1 = refine(df_p4_1[0])        
        df_p4_1 = refine_v2(df_p4_1)
        df_p4_1 = create_dataframe(df_p4_1)
    except:
        df_p4_1 = pd.DataFrame(columns=COLUMN_NAMES)


    res=pd.concat([df_p4_1, df_p4_2, df_p4_3], ignore_index = True)
    res['Assembly Constituency No and Name']=assm['Assembly Constituency No and Name'][0]
    res['Section No and Name']=assm['Section No and Name'][0]      
    return res

In [ ]:
def get_assembly_location(path):
    df_assm=read_pdf(path,pages="1",pandas_options={'header':None},area=[0,0,100,460])[0]
    df_assm=refine(df_assm)
    for i in range(len(df_assm)):
        df_assm['new'].iloc[i]=df_assm['new'].iloc[i].strip()
    df_assm.replace('', np.nan, inplace=True) 
    df_assm.dropna(inplace=True)
    asmbly=df_assm['new'].reset_index(drop=True)[1].split(':')[-1].strip()
    df_pol=read_pdf(path,pages="1",pandas_options={'header':None},area=[465,20,570,310])[0]
    polling=refine(df_pol)
    for i in range(len(polling)):
        polling['new'].iloc[i]=polling['new'].iloc[i].strip()
        temp=polling['new'].iloc[i]
        if ("Address of Polling" not in temp) and ("No and Name of Polling" not in temp) and (':' in temp or len(temp)<=5):
            polling['new'].iloc[i]=''
    polling.replace('', np.nan, inplace=True) 
    polling.dropna(inplace=True)
    pol=pd.DataFrame()
    pol['No. and Name of Polling Station']=[polling['new'].iloc[2].strip()]
    pol['Address of Polling Station']=[" ".join(list(map(lambda x: x.strip(),polling['new'].loc[4:].values)))]
    return asmbly,pol

In [ ]:
def get_pincode(path):
    df_pincode=read_pdf(path,pages="1",pandas_options={'header':None},area=[280,250,380,450])[0]
    df_pincode=refine(df_pincode)
    extr_dtls={'Main TownVillage':None,'Police Station':None,'Mandal':None,'Revenue Division':None,'District':None,'PinCode':None}
    for i in range(len(df_pincode)):
        if i%6==0:
            extr_dtls['Main TownVillage']=df_pincode.loc[i]['new'].split()[-1]
        elif i%6==1:
            extr_dtls['Police Station']=df_pincode.loc[i]['new'].split()[-1]
        elif i%6==2:
            extr_dtls['Mandal']=df_pincode.loc[i]['new'].split()[-1]
        elif i%6==3:
            extr_dtls['Revenue Division']=df_pincode.loc[i]['new'].split()[-1]
        elif i%6==4:
            extr_dtls['District']=df_pincode.loc[i]['new'].split()[-1]
        elif i%6==5:
            extr_dtls['PinCode']=df_pincode.loc[i]['new'].split()[-1]
    return extr_dtls

In [ ]:
def pdf_data_extractort(path):
    file=open(path, 'rb')
    readpdf = PyPDF2.PdfFileReader(file)
    totalpages = readpdf.numPages
    file.close()
    
    df1 = page_data_3(path)     
    df2 = page_data(4 , totalpages, path)
    res = pd.concat([df1, df2], ignore_index = True)
    asmbly,pol=get_assembly_location(path)
    df_pincode=get_pincode(path)
    res['Assembly Constituency is located']=asmbly
    print(pol['No. and Name of Polling Station'][0],pol['Address of Polling Station'][0])
    res['No. and Name of Polling Station']=pol['No. and Name of Polling Station'][0]
    res['Address of Polling Station']=pol['Address of Polling Station'][0]
    res['Main TownVillage']=df_pincode['Main TownVillage']
    res['Police Station']=df_pincode['Police Station']
    res['Mandal']=df_pincode['Mandal']
    res['Revenue Division']=df_pincode['Revenue Division']
    res['District']=df_pincode['District']
    res['PinCode']=df_pincode['PinCode']
    address=""
    for i in df_pincode.keys():
        if df_pincode[i]!= None:
            address+=df_pincode[i]+" "
    address=address.strip(" ")
    if len(re.findall(r'\d+',address))==0:
        res["PinCode_strip"]='NOT FOUND'
    else:
        res["PinCode_strip"]=re.findall(r'\d+',address)[-1]
    res['PinCode_slice']=address[-6:]
    return res

## Main Function
#### Give the path of input folder and output folder.
#### Get the logs of PDF in LOGS.csv if it is converted or not.

In [ ]:
import os
import PyPDF2
import time
i=0
logs={"OCRed_filename":[],"CSV_filename":[]}
path_feed=r".\Feed_folder"  #Folder with PDF to be coverted
output_path=r".\Output_folder" # Folder where you get the converted CSVs
entry_failed=[]
entry_success=[]
entries = os.listdir(path_feed)
for entry in entries:
    try:
        start_time = time.time()
        print("Start")
        print(path_feed+entry)
        file = open(path_feed+entry, 'rb')
        readpdf = PyPDF2.PdfFileReader(file)
        totalpages = readpdf.numPages
        print("Total pages===>",totalpages)
        file.close()
        pdf2=pdf_data_extractort(path_feed+entry)
        name=output_path+pdf2['Assembly Constituency is located'][0]+" "+pdf2['No. and Name of Polling Station'][0]+".csv"
        pdf2.to_csv(name)
        logs['OCRed_filename']+=[entry]
        logs['CSV_filename']+=[pdf2['Assembly Constituency is located'][0]+" "+pdf2['No. and Name of Polling Station'][0]+".csv"]
        entry_success.append(entry)
    except:
        entry_failed.append(entry)
    i+=1
    end_time = time.time()
    print("Completed")
    print("time taken=",end_time-start_time)
pd.DataFrame(logs).to_csv(output_path+"Logs_"+path_feed.split('\\')[-2]+".csv")
